# Car Price Prediction -- Random Forest Classification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df = pd.read_csv("cardekho_imputated.csv")
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Data Cleaning

In [3]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [4]:
df.drop("car_name", axis= 1, inplace=True)
df.drop("brand", axis= 1, inplace=True)

In [5]:
df.head()

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [6]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [7]:
num_features = [feature for feature in df.columns if df[feature].dtype != 'O'] #if not object then take it
print("Num of numerical features: ", len(num_features))
cat_features = [feature for feature in df.columns if df[feature].dtype == 'O']# take if its object
print("Num of categorical features: ", len(cat_features))
discrete_features = [feature for feature in num_features if len(df[feature].unique()) <= 25] # taking discrete features which have less than or equal to 25 unique values
print("Num of discrete features: ", len(discrete_features))
continuous_features = [feature for feature in num_features if feature not in discrete_features] # getting continuous features which have more than 25 unique values
print("Num of continuous feature : ", len(continuous_features))

Num of numerical features:  8
Num of categorical features:  4
Num of discrete features:  2
Num of continuous feature :  6


In [8]:
# Independent and dependent seperation
from sklearn.model_selection import train_test_split
X = df.drop(['selling_price'], axis = 1)
y = df['selling_price']

In [9]:
X.head()

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [10]:
y.head()

0    120000
1    550000
2    215000
3    226000
4    570000
Name: selling_price, dtype: int64

## Feature Encoding

In [11]:
len(df['model'].unique())

120

In [12]:
df['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [13]:
# we use lable encoding because the brands may be the part of price increase and decrement
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [14]:
X['model'].value_counts()

model
118    906
89     890
88     781
7      778
25     757
      ... 
8        1
15       1
52       1
75       1
55       1
Name: count, Length: 120, dtype: int64

In [15]:
X['model'].unique()

array([  7,  54, 118,  38, 100, 117,  96,  88,  97,  32,  29,  24,  16,
        60,  13,  89,  95,  30,  25,  14,  45,  64,  10,  84, 114,  63,
        59,  78,  85,  68,  11,  44,  98,  91,  73,  86,  23,  49,   1,
        19,  41,  62,  61,   0,   4,  92,  42,  83,  90,  58,  79,  39,
         5,  36,  74, 116,   2, 111, 106,  57,  26,  31,  22, 103,  77,
        46,  87,  70, 113,  34,  82,  99, 112,  93,  37, 101,  20,  40,
       115,  47,   3,  81,  33,  12, 107,  51,  28,  18,  65,  80,  94,
        56, 104,  71,  27,  17, 119,  53,  67, 105,  35, 109,  43,   6,
        66,  50,  48, 102, 110, 108,  72,   9,   8,  69,  21,  15,  76,
        52,  75,  55])

In [17]:
X['seller_type'].unique()

array(['Individual', 'Dealer', 'Trustmark Dealer'], dtype=object)

In [18]:
X['fuel_type'].unique()

array(['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric'], dtype=object)

In [19]:
X['transmission_type'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [20]:
#One hot encoding will create 120 extra columns which is waste of space and storage
len(X['seller_type'].unique()), len(X['fuel_type'].unique()), len(X['transmission_type'].unique())

(3, 5, 2)

In [21]:
#Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
onehot_features = ['seller_type', 'fuel_type', 'transmission_type'] 

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
# column transformer is a tool for appling different transformations to different columns of the dataset
# here we appling OneHotEncoding to categorical features and StandardScaler to numerical features
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, onehot_features), # it will apply OneHotEncoder to the categorical features
        ("StandardScaler", numeric_transformer, num_features), # it will apply StandardScaler to the numerical features
    ], 
    remainder='passthrough'#it will pass through to all those features which only have object datatype; 
)


In [22]:
X = preprocessor.fit_transform(X)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=54)
X_train.shape, X_test.shape

((11558, 18), (3853, 18))

In [24]:
X_train

array([[ 0.        ,  1.        ,  0.        , ..., -0.55087963,
        -0.27665384, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.47911321, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.17282578,
        -0.27665384, -0.40302241],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  1.32979434,
         0.45173447,  3.3116776 ],
       [ 0.        ,  1.        ,  0.        , ..., -0.55471774,
        -0.43582879, -0.40302241],
       [ 0.        ,  1.        ,  0.        , ...,  0.02099878,
         0.48221973, -0.40302241]])

In [25]:
y_train

8487      650000
11635     540000
1292      875000
9636     1075000
4181      490000
          ...   
5143     1900000
12047     190000
898       465000
7493      650000
7279      950000
Name: selling_price, Length: 11558, dtype: int64

### Model training and Model selection

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [27]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [28]:
#Beginning of Model training
models = {
    "Linear Regression": LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Nearest Neighbour": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])

    print("Model performance for Training set")
    print("- MAE: {:.4f}".format(model_train_mae))
    print("- RMSE score: {:.4f}".format(model_train_rmse))
    print("- R2 score: {:.4f}".format(model_train_r2))
    

    print("-----------------------------------------------------")

    print("Model performance for Test set")
    print("- MAE: {:.4f}".format(model_test_mae))
    print("- RMSE score: {:.4f}".format(model_test_rmse))
    print("- R2 score: {:.4f}".format(model_test_r2))

    print('=' *35)
    print('\n')


Linear Regression
Model performance for Training set
- MAE: 261345.6784
- RMSE score: 489774.6556
- R2 score: 0.6593
-----------------------------------------------------
Model performance for Test set
- MAE: 261981.6911
- RMSE score: 684423.7380
- R2 score: 0.5685


Lasso
Model performance for Training set
- MAE: 261343.9671
- RMSE score: 489774.6592
- R2 score: 0.6593
-----------------------------------------------------
Model performance for Test set
- MAE: 261979.5797
- RMSE score: 684423.6562
- R2 score: 0.5685


Ridge
Model performance for Training set
- MAE: 261314.9177
- RMSE score: 489775.2381
- R2 score: 0.6593
-----------------------------------------------------
Model performance for Test set
- MAE: 261940.1603
- RMSE score: 684443.9831
- R2 score: 0.5685


K-Nearest Neighbour
Model performance for Training set
- MAE: 97366.1338
- RMSE score: 258892.2490
- R2 score: 0.9048
-----------------------------------------------------
Model performance for Test set
- MAE: 128251.064

In [29]:
#Initialize few parameter for Hyperparameter tuning
knn_params = {"n_neighbors": [2,3,10,20,40,50]}
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split":[2,8,15,20],
             "n_estimators" : [100,200,500,1000]
             }

In [30]:
#Models list for hyperparameter tuning
randomcv_models = [('KNN', KNeighborsRegressor(), knn_params), ('RF', RandomForestRegressor(), rf_params),]

In [31]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=100, cv = 3,verbose=2, n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"-------------------Best params for {model_name} ------------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
-------------------Best params for KNN ------------------------
{'n_neighbors': 3}
-------------------Best params for RF ------------------------
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}


In [32]:
models = {
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, min_samples_split=2, max_features=8, max_depth=None),
    "K-Neighbours Regressor": KNeighborsRegressor(n_neighbors=3, n_jobs=-1)
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])


    print("Model performance for Training set")
    print("- MAE: {:.4f}".format(model_train_mae))
    print("- RMSE score: {:.4f}".format(model_train_rmse))
    print("- R2 score: {:.4f}".format(model_train_r2))
    

    print("-----------------------------------------------------")

    print("Model performance for Test set")
    print("- MAE: {:.4f}".format(model_test_mae))
    print("- RMSE score: {:.4f}".format(model_test_rmse))
    print("- R2 score: {:.4f}".format(model_test_r2))

    print('=' *35)
    print('\n')

Random Forest Regressor
Model performance for Training set
- MAE: 35661.4153
- RMSE score: 99196.4826
- R2 score: 0.9860
-----------------------------------------------------
Model performance for Test set
- MAE: 98629.7375
- RMSE score: 480836.0864
- R2 score: 0.7870


K-Neighbours Regressor
Model performance for Training set
- MAE: 84488.6118
- RMSE score: 232296.5026
- R2 score: 0.9234
-----------------------------------------------------
Model performance for Test set
- MAE: 134309.6180
- RMSE score: 584557.0884
- R2 score: 0.6852


